# From "Chi by Eye" to MCMC: An Introduction to Estimating Errors in Astronomy
## Starfish School 2022
### By: Joshua S. Speagle & Ting S. Li

In [ ]:
# only necessary if you're running Python 2.7 or lower
from __future__ import print_function, division
from six.moves import range

# import matplotlib and define our alias
from matplotlib import pyplot as plt

# plot figures within the notebook rather than externally
%matplotlib inline

# numpy
import numpy as np

# scipy 
import scipy

# Overview

This notebook will try to give an introduction to statistics and error estimation in astronomy. Through the notebook, we will go through a simple **linear regression** example that appears simple at first glance, but actually has a surprising amount of depth and is applicable in a bunch of different domains. 

Most importantly, it provides an accessible way to get a handle on several big concepts in **Bayesian Inference**, including:
- defining objective functions,
- probabilities and likelihoods, 
- estimating parameters,
- the concept of priors, and
- marginalizing over uncertainties.

By the end of the talk, we'll have built up to the concepts behind **Markov Chain Monte Carlo (MCMC)**, which some of you may have heard of and which has become pretty ubiquitous in astronomy.

# Data

Let's generate some mock data below.

In [ ]:
# set random number seed to ensure reproducibility
seed = 123
rstate = np.random.RandomState(seed)

In [ ]:
def gen_mock_data(m=0.875, b=2.523, s=0.523, unc=[0.2, 0.6], N=20):
    """
    Generate `N` mock data points from the line
    with slope `m`, intercept `b`, and
    intrinsic scatter `s` with measurement uncertainties
    uniformly distributed between the values in `unc` using
    a random number generator with the starting `seed`.
    
    """
        
    # generate synthetic data
    x = np.sort(3. * rstate.rand(N))  # x values
    y_int = m * x + b  # underlying line
    y = np.random.normal(y_int, s)  # with intrinsic scatter
    yerr = np.random.uniform(unc[0], unc[1], N)  # measurement errors
    yobs = np.random.normal(y, yerr)
    
    return x, yobs, yerr

In [ ]:
# generate mock data
x, y, ye = gen_mock_data()

In [ ]:
# plot data
plt.figure(figsize=(14, 8))
plt.errorbar(x, y, yerr=ye, fmt="ko", ecolor='gray', capsize=0)
plt.xlabel('x')
plt.ylabel('y')
plt.tight_layout()

In [ ]:
np.savetxt('Session5_data.txt', np.c_[x, y, ye])

# Problem

Our starting goal here is going to be pretty simple: **try to determine a simple linear relationship**. 

In other words, we want a model like

$$ y = mx + b $$

where $m$ is the slope and $b$ is the $y$-intercept. We will slowly make this model more complicated throughout the talk as we try and model additional features of the data.

# "Ocular Least Squares" / "Chi by Eye"

To start, let's ignore all attempts to make this quantitative. The human brain is actually quite good at fitting models to data, so we can do an **"ocular least squares"** process (based on the "ordinary least squares" approach we will define shortly) or **"chi by eye"** process (based on the "chi-squared" statistic we will define later) and just try and benchmark what looks like a reasonable fit.

In [ ]:
# best estimates of slope and intercept
m_1, b_1 = 1.25, 2.

# plot results
plt.figure(figsize=(14, 8))
plt.errorbar(x, y, yerr=ye, fmt="ko", ecolor='gray', 
             capsize=0, label='Data')  # plot data
plt.plot(x, m_1 * x + b_1, lw=4, alpha=0.7, color='red',
         label='by eye')  # plot chi-by-eye
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()

Our guess probably looks "reasonable", but you might not feel so good about it. Why? 

Can you say *why* it's the best fit? How about uncertainties on the slope and intercept?

Part of the difficulty here is that there is no quantitative metric for what makes a fit "good". We will now try to rectify this problem.

# Objective/Loss/Cost Function

One way to define whether a fit is any good is to see how well it actually *predicts* the data it is supposed to be fitting. One way to quantify this involves looking at the residuals

$$ |\Delta y_i| = |y_{i, {\rm pred}} - y_{i}| $$

where $i$ indicates the $i$th datapoint, $y_{i, {\rm pred}} = m x_i + b$ is the prediction from the model and $y_{i}$ is the observed (noisy) value. We are interested in the absolute value because what *probably* matters is the total error, not whether it's positive or negative. *(Note that we are currently ignoring the errors.)*

We would ideally like to have a model that has the smallest residuals for all the points. Let's therefore define a **loss function** (also often called an **objective function** or a **cost function**):

$$ L(m, b | \{ (x_1, y_1), \dots, (x_N, y_N) \}) = \sum_{i=1}^{N} |\Delta y_i|^p $$

where the $|$ line now indicates "given" or "conditioned on". In other words, this equation reads:

> "What is the loss of a particular slope $m$ and intercept $b$ given the data $\{ (x_1, y_1), \dots, (x_N, y_N) \}$?"

$p$ is a power that all residuals are raised to that control how sensitive the loss function is to large and small residuals. Common values for $p$ are often $1 \leq p \leq 2$, with $p=2$ (squaring the residuals) being the most common.

In [ ]:
# Loss function
def loss(theta, p=2., x=x, y=y):
    """A simple loss function as a function of parameters `theta`."""
    
    m, b = theta  # reassign parameters
    ypred = m * x + b
    resid = np.abs(ypred - y)
    
    return np.sum(resid**p)

In [ ]:
# change m at fixed b
mgrid = np.linspace(0, 2, 100)
loss_m = np.array([loss([m, b_1], p=2.) for m in mgrid])
    
# change b at fixed m
bgrid = np.linspace(0, 6, 100)
loss_b = np.array([loss([m_1, b], p=2.) for b in bgrid])


# plot results
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.plot(mgrid, loss_m, lw=3, color='firebrick')  # varying m given b
plt.xlabel('m (fixed b)')
plt.ylabel('Loss')
plt.subplot(1, 2, 2)
plt.plot(bgrid, loss_b, lw=3, color='navy')  # varying b given m
plt.xlabel('b (fixed m)')
plt.ylabel('Loss')
plt.tight_layout()

# Optimization

To find the best fit, we want to **minimize our loss** (alternately, **optimize our objective**).

We will use the `minimize` function from `scipy.optimize` (see [documentation](https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.optimize.minimize.html)) to get the best-fit parameters `theta` based on our loss function `loss` and quantify the difference relative to our initial results.

In [ ]:
from scipy.optimize import minimize

# minimize the loss function
results = minimize(loss, [m_1, b_1])

# print the results
print(results)

In [ ]:
# get best-fit result
m_2, b_2 = results['x']

# plot results
plt.figure(figsize=(14, 8))
plt.errorbar(x, y, yerr=ye, fmt="ko", ecolor='gray', 
             capsize=0, label='Data')  # plot data
plt.plot(x, m_1 * x + b_1, lw=4, alpha=0.4, color='red',
         label='by eye')  # plot chi-by-eye
plt.plot(x, m_2 * x + b_2, lw=4, alpha=0.7, color='dodgerblue',
         label='loss')  # plot minimum loss
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()

In [ ]:
# quantifying the improvement
print('Change in Loss:', loss([m_1, b_1]) - loss([m_2, b_2]))

# Incorporating Errors with Chi-Square

So far, we haven't tried to incorporate **errors** into our linear regression model. We probably want our fit to take these into account, especially if the errors can change from point to point! Points with large errors probably should be "down-weighted" in the fit since they are more uncertain. We can accomplish this be defining the **error-normalized resuidual** $\chi$:

$$ \chi_i = \left|\frac{\Delta y_i}{\sigma_i}\right| $$

where $\sigma_i$ is the measurement error associated with $y_i$. This should make some intuitive sense: we are just normalizing the residual by the error, so now we are measuring how discrepant the prediction is in units of the error bar for each point.

We can now define the **chi-square statistic**:

$$ \chi^2(m, b | \{ (x_1, y_1, \sigma_1), \dots, (x_N, y_N, \sigma_N) \}) = \sum_{i=1}^{N} \chi^2_i $$

You may have heard this term being used before to describe the quality of a fit to data, since the $\chi^2$ statistic is used quite often in the astronomical literature!

In [ ]:
# chi2 function
def chi2(theta, x=x, y=y, ye=ye):
    """Chi-square as a function of parameters `theta`."""
    
    m, b = theta  # reassign parameters
    ypred = m * x + b
    resid = ypred - y
    
    return np.sum(resid**2 / ye**2)

In [ ]:
# minimize chi2
results = minimize(chi2, [m_2, b_2])
print(results)

In [ ]:
# get best-fit result
m_3, b_3 = results['x']

# plot results
plt.figure(figsize=(14, 8))
plt.errorbar(x, y, yerr=ye, fmt="ko", ecolor='gray', 
             capsize=0, label='Data')  # plot data
plt.plot(x, m_1 * x + b_1, lw=4, alpha=0.4, color='red',
         label='by eye')  # plot chi-by-eye
plt.plot(x, m_2 * x + b_2, lw=4, alpha=0.4, color='dodgerblue',
         label='loss')  # plot minimum loss
plt.plot(x, m_3 * x + b_3, lw=4, alpha=0.7, color='darkviolet',
         label='chi2')  # plot minimum chi2
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()

In [ ]:
# quantifying the improvement
print('Change in chi2:', 
      chi2([m_1, b_1]) - chi2([m_3, b_3]), 
      chi2([m_2, b_2]) - chi2([m_3, b_3]))

# From Loss Functions to Probabilistic Models

We now want to consider an additional source of uncertainty: some amount of **intrinsic scatter** in the fitted relationship. In other words, in addition to the scatter $\sigma_i$ caused by the observational uncertainties, we also want to add on an additional scatter $s_i = s$ that is the same for all points.

How might we do so? Well, maybe we could try something like

$$\chi = \left|\frac{\Delta y_i}{\sigma_i + s}\right|$$

Notice, however, that $\chi^2$ will always get smaller as $s$ increases. Whoops! 

Can we add on a penalty to disfavor large values? Maybe something like

$$ \chi^2(m, b, s | \{ (x_1, y_1, \sigma_1), \dots, (x_N, y_N, \sigma_N) \}) = s^2 + \sum_{i=1}^{N} \chi^2_i $$

could work. We could even argue for reasons why it might be reasonable!

This basic result, however, reveals a *fundamental problem* with how we're approached fitting a line so far: we haven't actually defined an underlying **probabilistic model** for the data.

# Likelihoods

All models start with trying to understand what I like to call the **data generating process**. In other words, if we wanted to simulate data based on an input set of parameters, how would we do so?

## Probability Density Functions (PDFs)

To do so, let's start with the observed $y$ values. We have **observational uncertainties**, but what do those *mean*? 

Well, in general it means we assume the observed data follow a **Normal distribution** (also called a **Gaussian distribution**) such that the probability that $y_i$ is any particular value follows a **probability density function (PDF)** of the form

$$ P(y_i|y_{i,{\rm true}}, \sigma_i) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left[-\frac{1}{2}\frac{(y_{i, {\rm true}} - y_i)^2}{\sigma_i^2}\right] $$

Since our model for the data is $y_{i, {\rm true}} = m x_i + b$, this then gives

$$ P(y_i|m, b, x_i, \sigma_i) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left[-\frac{1}{2}\frac{(m x_i + b - y_i)^2}{\sigma_i^2}\right] $$

Although it is beyond the scope of this notebook to prove, it can be shown that assuming that the intrinsic scatter $s$ is also Gaussian gives the modified result:

$$ P(y_i|m, b, s, x_i, \sigma_i) = \frac{1}{\sqrt{2\pi(\sigma_i^2 + s^2)}} \exp\left[-\frac{1}{2}\frac{(m x_i + b - y_i)^2}{\sigma_i^2 + s^2}\right] $$

## Combining Data

The probability of *independent* events $A$, $B$, and $C$ all occuring is just the product of their individual probabilities:

$$ P(A, B, C) = P(A) P(B) P(C) $$

The same holds true for data points: if each data point is an independent observation, the total probability is just the individual probabilities for each data point multiplied together. This gives

$$ P(y_1, \dots, y_N|m, b, s, (x_1, \sigma_1), \dots, (x_N, \sigma_N)) = \prod_{i=1}^{N} P(y_i|m, b, s, x_i, \sigma_i) $$

Since the probabilities can get really small really fast, for numerical stability this is almost always re-written in logarithmic form:

$$ \ln P(y_1, \dots, y_N|m, b, s, (x_1, \sigma_1), \dots, (x_N, \sigma_N)) = \sum_{i=1}^{N} \ln P(y_i|m, b, s, x_i, \sigma_i) $$

Notice that, if we explicitly substitute in the Gaussian probability density function, we get this intriguing result:

$$ \ln P(y_1, \dots, y_N|m, b, s, (x_1, \sigma_1), \dots, (x_N, \sigma_N)) = -\frac{1}{2} \sum_{i=1}^{N} \frac{(mx_i + b - y_i)^2}{\sigma_i^2 + s^2} + \ln(2\pi(\sigma_i^2 + s^2)) $$

The first term here looks a lot like our original $\chi^2$ expression, except now with the modified errors. And the second term now looks a lot like a penalty term that disfavors larger $s$ values! 

*By explicitly writing out a model, we naturally accomplish our original objective!* A lot of neat results often happen this way -- when you're struggling with trying to fit a dataset, going back to the fundamentals often can reveal a lot of neat results like this.

# Maximum-Likelihood Estimate (MLE)

The "best-fit" parameters now can be defined as those that **maximize the likelihood** (or, equivalently, **minimize the negative of the likelihood**). Since logarithms don't change the maximum or minimum, we often use the log-likelihoods in practice.

In [ ]:
def negloglike(theta, x=x, y=y, ye=ye):
    """(Negative) log-likelihood as a function of parameters `theta`."""
    
    m, b, s = theta  # reassign parameters
    ypred = m * x + b
    resid = ypred - y
    chi2 = resid**2 / (ye**2 + s**2)  # chi2 term
    const = np.log(2 * np.pi * (ye**2 + s**2))  # normalization/penalty term
    logl = -0.5 * np.sum(chi2 + const)
    
    return -logl

In [ ]:
# minimize negative log-likelihood
results = minimize(negloglike, [m_3, b_3, 0.5])
results_mle = results
print(results)

In [ ]:
# get best-fit result
m_4, b_4, s_4 = results['x']

# plot results
plt.figure(figsize=(14, 8))
plt.errorbar(x, y, yerr=ye, fmt="ko", ecolor='gray', 
             capsize=0, label='Data')  # plot data
plt.plot(x, m_1 * x + b_1, lw=4, alpha=0.4, color='red',
         label='by eye')  # plot chi-by-eye
plt.plot(x, m_2 * x + b_2, lw=4, alpha=0.4, color='dodgerblue',
         label='loss')  # plot minimum loss
plt.plot(x, m_3 * x + b_3, lw=4, alpha=0.4, color='darkviolet',
         label='chi2')  # plot minimum chi2
plt.plot(x, m_4 * x + b_4, lw=4, alpha=0.7, color='darkorange', 
         label='MLE')  # plot MLE
[plt.fill_between(x, m_4 * x + b_4 + s_4 * n, m_4 * x + b_4 - s_4 * n, 
                  alpha=0.02, color='darkorange')
 for n in np.linspace(0, 2, 20)]  # plot MLE uncertainty estimates
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()

In [ ]:
# quantifying the improvement
print('Change in loglike:', 
      negloglike([m_3, b_3, 0]) - negloglike([m_4, b_4, s_4]))

# Bayesian Inference

We now have a working model that can describe the underlying probabilistic processes that generate the data. However, we're still missing one small thing. 

In our earlier notation,

$$ P(A | B) $$

describes the probability of $A$ *given* (i.e. conditioned on) $B$. So our likelihood

$$ P(y_1, \dots, y_N | m, b, s, (x_1, \sigma_1), \dots, (x_N, \sigma_N)) $$

describes something similar: the probability of seeing the observed *data* $\{ y_1, \dots, y_N\}$ given the underlying model parameters $m$, $b$, and $s$ along with the $x_i$ and $\sigma_i$ values. 

What we *want*, however, is this expression:

$$ P(m, b, s | (x_1, y_1, \sigma_1), \dots, (x_N, y_N, \sigma_N)) $$

This now describes the **probability of our *model parameters* given the data**.

# Bayes' Theorem

Getting at this distribution requires exploiting the fact that probabilities can factor. This allows us to rewrite:

$$ P(A, B) = P(A|B)P(B) = P(B|A)P(A) = P(B, A) $$

Rearranging then gives:

$$ P(B|A) = \frac{P(A|B) P(B)}{P(A)} $$

If we replace $A = {\rm data}$ and $B = {\rm parameters}$, we are left with what's known as **Bayes' Theorem**:

$$ P({\rm parameters} \,|\, {\rm data}) = \frac{P({\rm data}\,|\,{\rm parameters}) \, P({\rm parameters})}{P({\rm data})} $$

Pulling apart this expression:
- The term $P({\rm data}\,|\,{\rm parameters})$, is the **likelihood** that we have been working with already and describes the probability of seeing the data given the model. 
- The term $P({\rm parameters})$ is known as the **prior**: it characterizes our prior knowledge about the parameters is question without seeing the data. 
- The term $P({\rm data})$ is known as the **evidence** (or marginal likelihood). Since this is just a constant, most often we can just ignore it. (It is useful though! Feel free to ask me about it.)
- Finally, the left-hand side $P({\rm parameters} \,|\, {\rm data})$ is known as the **posterior** since it combines the prior and the likelihood together.

# *Maximum-a-Posteriori* (MAP) Estimate

Finally, we can now find our parameter estimates with the highest log-posterior values, known as the **maximum-a-posteriori** (MAP) estimate.

In [ ]:
# define a Gaussian prior over each parameter
prior_means = np.array([1., 3., 0.5])  # m, x, b
prior_stds = np.array([0.25, 0.5, 0.15])  # m, x, b

def neglogprior(theta, mean=prior_means, std=prior_stds):
    """(Negative) log-prior as a function of parameters `theta`."""
    
    chi2 = (theta - mean)**2 / std**2
    const = np.log(2. * np.pi * std**2)
    logp = -0.5 * np.sum(chi2 + const)
    
    return -logp

def neglogpost(theta, x=x, y=y, ye=ye, mean=prior_means, std=prior_stds):
    """(Negative) log-posterior as a function of parameters `theta`."""
    
    m, b, s = theta  # reassign parameters
    logp = -neglogprior(theta, mean=mean, std=std)  # prior
    logl = -negloglike(theta, x=x, y=y, ye=ye)  # likelihood
    
    return -(logl + logp)  # posterior

In [ ]:
# minimize negative log-posterior
results = minimize(neglogpost, [m_4, b_4, s_4])
results_map = results
print(results)

In [ ]:
# get best-fit result
m_5, b_5, s_5 = results['x']

# plot it against previous results
plt.figure(figsize=(14, 8))
plt.errorbar(x, y, yerr=ye, fmt="ko", ecolor='gray', 
             capsize=0, label='Data')  # plot data
plt.plot(x, m_1 * x + b_1, lw=4, alpha=0.4, color='red',
         label='by eye')  # plot chi-by-eye
plt.plot(x, m_2 * x + b_2, lw=4, alpha=0.4, color='dodgerblue',
         label='loss')  # plot minimum loss
plt.plot(x, m_3 * x + b_3, lw=4, alpha=0.4, color='darkviolet',
         label='chi2')  # plot minimum chi2
plt.plot(x, m_4 * x + b_4, lw=4, alpha=0.4, color='darkorange', 
         label='MLE')  # plot MLE (no priors)
plt.plot(x, m_5 * x + b_5, lw=4, alpha=0.7, color='gray', 
         label='MAP')  # plot MAP (including priors)
[plt.fill_between(x, m_5 * x + b_5 + s_5 * n, m_5 * x + b_5 - s_5 * n, 
                  alpha=0.02, color='gray')
 for n in np.linspace(0, 2, 20)]  # plot MAP uncertainty estimates
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.tight_layout()

# Parameter Uncertainties

Our probability distributions can be used to estimate more than just best-fit parameters. They also give us estimates of the uncertainties by telling us just how much more "probable" one set of parameters is compared to another. Exploring this uncertainty is hard though, since a function like

$$ \ln P(m, b, s | y_1, \dots, y_N, (x_1, \sigma_1), \dots, (x_N, \sigma_N)) $$

doesn't just come with labels attached for how the (log-)probability changes as a function of the parameters!

# Inverse Hessian (Gaussian) Approxmation

While exploring this fully is beyond the scope of this introduction, one neat result of using a properly defined probabilistic model is that `scipy.minimize` provides estimates of the parameter errors! This can be accessed via the `hess_inv` item in the output results dictionary. We can interpret this as essentially a "Gaussian" approximation: these are what the errors would be if we assume the unknown distribution was roughly Gaussian centered around the best-fit parameters.

In [ ]:
n = int(1e6)  # number of samples to draw

# generate prior samples
m_prior, b_prior, s_prior = np.random.multivariate_normal(prior_means, 
                                                          np.diag(prior_stds**2), 
                                                          size=n).T

# generate likelihood samples
m_like, b_like, s_like = np.random.multivariate_normal(results_mle['x'], 
                                                       results_mle['hess_inv'], 
                                                       size=n).T

# generate posterior samples
m_post, b_post, s_post = np.random.multivariate_normal(results_map['x'], 
                                                       results_map['hess_inv'], 
                                                       size=n).T

# define bins
bins = [np.linspace(0, 2, 100),  # bins (m)
        np.linspace(1, 5, 100),  # bins (b)
        np.linspace(0, 1, 100)]  # bins (s)

# generate 1-D histograms of priors vs posteriors
plt.figure(figsize=(15, 10))
# slope (m)
plt.subplot(2, 3, 1)
plt.hist(m_prior, bins[0], density=True, label='prior',
         color='dodgerblue', alpha=0.6)
plt.hist(m_like, bins[0], density=True, label='like',
         color='orange', alpha=0.6)
plt.hist(m_post, bins[0], density=True, label='post',
         color='firebrick', alpha=0.6)
plt.xlim(bins[0][0], bins[0][-1])
plt.xlabel('m')
plt.ylabel('Probability')
plt.yticks([])
plt.legend()
# y-intercept (b)
plt.subplot(2, 3, 2)
plt.hist(b_prior, bins[1], density=True, label='prior',
         color='dodgerblue', alpha=0.6)
plt.hist(b_like, bins[1], density=True, label='like',
         color='orange', alpha=0.6)
plt.hist(b_post, bins[1], density=True, label='post',
         color='firebrick', alpha=0.6)
plt.xlim(bins[1][0], bins[1][-1])
plt.xlabel('b')
plt.ylabel('Probability')
plt.yticks([])
# scatter (s)
plt.subplot(2, 3, 3)
plt.hist(s_prior, bins[2], density=True, label='prior',
         color='dodgerblue', alpha=0.6)
plt.hist(s_like, bins[2], density=True, label='like',
         color='orange', alpha=0.6)
plt.hist(s_post, bins[2], density=True, label='post',
         color='firebrick', alpha=0.6)
plt.xlim(bins[2][0], bins[2][-1])
plt.xlabel('s')
plt.ylabel('Probability')
plt.yticks([])
plt.tight_layout()

# generate 2-D histograms of posteriors

plt.subplot(2, 3, 4)
plt.hist2d(m_post, b_post, [bins[0], bins[1]],
           cmap='Reds', alpha=0.8)
plt.hist2d(m_like, b_post, [bins[0], bins[1]],
           cmap='Oranges', alpha=0.6)
plt.hist2d(m_prior, b_prior, [bins[0], bins[1]],
           cmap='Blues', alpha=0.3)
plt.xlabel('m')
plt.ylabel('b')
plt.subplot(2, 3, 5)
plt.hist2d(b_post, s_post, [bins[1], bins[2]],
           cmap='Reds', alpha=0.8)
plt.hist2d(b_like, s_post, [bins[1], bins[2]],
           cmap='Oranges', alpha=0.6)
plt.hist2d(b_prior, s_prior, [bins[1], bins[2]],
           cmap='Blues', alpha=0.3)
plt.xlabel('b')
plt.ylabel('s')
plt.subplot(2, 3, 6)
plt.hist2d(s_post, m_post, [bins[2], bins[0]],
           cmap='Reds', alpha=0.8)
plt.hist2d(s_like, m_post, [bins[2], bins[0]],
           cmap='Oranges', alpha=0.6)
plt.hist2d(s_prior, m_prior, [bins[2], bins[0]],
           cmap='Blues', alpha=0.3)
plt.xlabel('s')
plt.ylabel('m')
plt.tight_layout()

One way to tyy and visualize all of this is through the use of a **corner plot** (also known as a triangle plot). The `corner` package (see [documentation](https://corner.readthedocs.io/en/latest/index.html)) provides an easy to use interface to visualize the density of samples in 1-D and 2-D, like above, in one easy place.

In [ ]:
import corner

# plot posterior
corner.corner(np.c_[m_post, b_post, s_post],  # collect samples into N x 3 array
              bins=50,  # bins for histogram
              labels=['m', 'b', 's'],
              show_titles=True, quantiles=[0.16, 0.84],  # show median and uncertainties
              truths=results_map['x'],  # plot MAP
              color='firebrick', truth_color='black',  # add some colors
              **{'plot_datapoints': False, 'fill_contours': True});  # change some default options

# Markov Chain Monte Carlo (MCMC)

Finally, we get to **Markov Chain Monte Carlo (MCMC)**. For this, I'm going to focus on generating a few plots highlighted in a paper I wrote a while back ([Speagle 2019](https://arxiv.org/pdf/1909.12313.pdf)) to showcase how it works.

What we will use is an implementation provided as part of the `emcee` package (see [documentation](https://emcee.readthedocs.io/en/stable/)).

In [ ]:
def logpost(theta, x=x, y=y, ye=ye, mean=prior_means, std=prior_stds):
    """(Negative) log-posterior as a function of parameters `theta`."""
    
    m, b, s = theta  # reassign parameters
    logp = -neglogprior(theta, mean=mean, std=std)  # prior
    logl = -negloglike(theta, x=x, y=y, ye=ye)  # likelihood
    
    return (logl + logp)  # posterior

In [ ]:
import emcee

ndim = 3  # number of parameters
nwalkers = 100  # number of "walkers" or "chains" to run

# initialize starting positions from our initial approximation
p0 = np.c_[m_post, b_post, s_post][:nwalkers]

# initialize our sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, logpost)

# initial "burn-in" run
state = sampler.run_mcmc(p0, 1000, progress=True)
sampler.reset()

# final "production" run
state = sampler.run_mcmc(state, 10000, progress=True)

# get final chains
samples = sampler.get_chain()

In [ ]:
# plot a few chains
plt.figure(figsize=(14, 9))
# slope (m)
plt.subplot(3, 1, 1)
[plt.plot(samples[:, i, 0], alpha=0.5) for i in range(4)]
plt.xlim([0, 5000])
plt.xlabel('Iteration')
plt.ylabel('m')
# intercept (b)
plt.subplot(3, 1, 2)
[plt.plot(samples[:, i, 0], alpha=0.5) for i in range(4)]
plt.xlabel('Iteration')
plt.ylabel('b')
plt.xlim([0, 5000])
# scatter (s)
plt.subplot(3, 1, 3)
[plt.plot(samples[:, i, 0], alpha=0.5) for i in range(4)]
plt.xlim([0, 5000])
plt.xlabel('Iteration')
plt.ylabel('s')
plt.tight_layout()

In [ ]:
# plot posterior
corner.corner(samples.reshape(-1, ndim),  # collect samples into N x 3 array
              bins=50,  # bins for histogram
              show_titles=True, quantiles=[0.16, 0.84],  # show median and uncertainties
              labels=['m', 'b', 's'],
              truths=results_map['x'],  # plot MAP
              color='darkviolet', truth_color='black',  # add some colors
              **{'plot_datapoints': False, 'fill_contours': True});  # change some default options